- Scraping the menu items from https://pizzaonline.dominos.co.in/
- Selecting Prices for each pizza with different crusts and sizes
- Period of Data Extraction : **'2021-12-15 23:19:03'**

In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [2]:
path = Service('/usr/local/bin/chromedriver')
op = webdriver.ChromeOptions()
# op.add_argument('headless');
driver = webdriver.Chrome(service=path,
                          options=op)

In [3]:
wait = WebDriverWait(driver, 1)

closed_window_1 = False
closed_window_2 = False

size,choose_size,current_size,current_price,current_crust=[0]*5

def close_popups():
    global closed_window_1,closed_window_2
    if closed_window_1==False:
        try:
            close_1 = wait.until(EC.presence_of_element_located((By.XPATH, '// *[ @ id = "close-icon"]')))
            close_1.click()
            closed_window_1 = True
            print("Closed Prompt 1")

        except:
            pass
        try:
            close_1 = driver.switch_to.alert
            close_1.dismiss()
            closed_window_1 = True
            print("Closed Prompt 1")
        except:
            pass

    if closed_window_2==False:
        try:
            close_2 = wait.until(EC.presence_of_element_located(
                (By.XPATH, '// *[ @ id = "desktopBannerWrapped"] / div / div[3] / div[1] / button[1]')))
            close_2.click()
            closed_window_2 = True
            print("Closed Prompt 2")
        except:
            pass

def perform_assignment(RHS):
    try:
        x=RHS
        # print("LHS=RHS")
    except:
        close_popups()
        x=RHS
        # print("Post exception : LHS=RHS")

    return x

In [4]:
driver.get("https://pizzaonline.dominos.co.in/menu")
i=1
data=[]
category={2 : 'Best Sellers',
          4 : 'Veg Pizza',
          6 : 'Non-Veg Pizza',
          8 : 'Chicken Lovers Pizza',
          12 : 'Meals & Combos',
          14 : 'Pizza Mania',
          16 : 'Speciality Chicken',
          18 : 'Sides',
          20 : 'Beverages',
          22 : 'Desserts'
          }
for s in range(2,23,2):
    section  = driver.find_elements(By.XPATH,f'//*[@id="mn-lft"]/div[{s}]/div/div')
    print("Pizza Section No : ", i)
    print("Actual Section No : ",s)
    print("No. of items : ",len(section))

    for p in range(1,len(section)+1):
        close_popups()
        xpath = f'//*[@id="mn-lft"]/div[{s}]/div/div[{p}]/'
        name  = driver.find_element(By.XPATH,xpath+'div/div/div[2]/div[1]/span[1]').text
        desc  = driver.find_element(By.XPATH,xpath+'div/div/div[2]/div[1]/span[2]').text

        # FOR ALL PIZZAS
        if s in [2,4,6,8]:
            for size_index in [3,2,1]: 
                # Small Pizzas cover all crusts so have been put at the end for element recovery
                # Contrast - Large pizzas cover the least crusts so have been put at the start
                
                for crust_index in [1,2,3,4,5]:
                    try:
                        # Click on size dropdown
                        size=perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[2]/div[2]/div[1]/div[2]/div/div/div[1]'))))
                        driver.execute_script("arguments[0].click();", size)

                        # Choose size
                        choose_size = perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+f'div/div/div[2]/div[2]/div[1]/div[2]/div/div/ul/li[{size_index}]/span/span[1]/span[1]'))))
                        driver.execute_script("arguments[0].click();", choose_size)
                        
                        # Click on crust dropdown
                        crust=perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[2]/div[2]/div[2]/div[2]/div/div/div[1]'))))
                        driver.execute_script("arguments[0].click();", crust)
                        
                        # Choose size
                        choose_crust = perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+f'div/div/div[2]/div[2]/div[2]/div[2]/div/div/ul/li[{crust_index}]/span/span[1]'))))
                        driver.execute_script("arguments[0].click();", choose_crust)
                        

                        # Select active state of size,crust & the updated price
                        current_size = perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[2]/div[2]/div[1]/div[2]/div/div/div[1]'))).text)
                        current_crust = perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[2]/div[2]/div[2]/div[2]/div/div/div[1]'))).text)
                        current_price=perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[1]/div/div[3]/span'))).text)
                        
                        print(f"Added :  Pizza no {p}    Category {s}    Size {size_index}")
                        dic = {'Item': name,
                               'Category': category[s],
                               'Description': desc,
                               'Price': current_price,
                               'Size': current_size,
                               'Crust': current_crust}
                        print(f"Added :  Pizza    {dic['Item']}    Category{dic['Category']}       Size {dic['Size']}")

                        data.append(dic)
                    except Exception as e:
                        pass
                    
        # FOR PIZZA MANIA
        if s==14:
            for crust_index in [1,2]:
                try:
                    # Click on crust dropdown
                    crust=perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[2]/div[2]/div[2]/div[2]/div/div/div[1]'))))
                    driver.execute_script("arguments[0].click();", crust)
                        
                    # Choose size
                    choose_crust = perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+f'div/div/div[2]/div[2]/div[2]/div[2]/div/div/ul/li[{crust_index}]/span/span[1]'))))
                    driver.execute_script("arguments[0].click();", choose_crust)
                    
                    # Select active state of size,crust & the updated price
                    current_size = "Regular"
                    current_crust = perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[2]/div[2]/div[2]/div[2]/div/div/div[1]'))).text)
                    current_price=perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[1]/div/div[3]/span'))).text)
                    
                    print(f"Added :  Pizza no {p}    Category {s}    Size {size_index}")
                    dic = {'Item': name,
                           'Category': category[s],
                           'Description': desc,
                           'Price': current_price,
                           'Size': current_size,
                           'Crust': current_crust}
                    print(f"Added :  Pizza    {dic['Item']}    Category{dic['Category']}       Size {dic['Size']}")

                    data.append(dic)
                except Exception as e:
                    pass
                
        # NON-PIZZA ITEMS
        if s in [12,16,18,20,22]:
            # Select active state of size,crust & the updated price
            current_size = "N/A"
            current_crust= "N/A"
            current_price=perform_assignment(wait.until(EC.presence_of_element_located((By.XPATH,xpath+'div/div/div[1]/div/div[3]/span'))).text)
                        
            
            dic = {'Item': name,
                   'Category': category[s],
                   'Description': desc,
                   'Price': current_price,
                   'Size': current_size,
                   'Crust': current_crust}
            
            print(f"Added :  Item    {dic['Item']}    Category{dic['Category']}       Size {dic['Size']}")

            data.append(dic)
            
            
            
            
        
    print("------------------------------")
    i+=1

df = pd.DataFrame(data)
print(df)


time.sleep(5)

driver.quit()




Pizza Section No :  1
Actual Section No :  2
No. of items :  8
Closed Prompt 2
Added :  Pizza no 1    Category 2    Size 3
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Large
Added :  Pizza no 1    Category 2    Size 2
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Medium
Added :  Pizza no 1    Category 2    Size 2
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Medium
Added :  Pizza no 1    Category 2    Size 2
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Medium
Added :  Pizza no 1    Category 2    Size 2
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Medium
Added :  Pizza no 1    Category 2    Size 1
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Regular
Added :  Pizza no 1    Category 2    Size 1
Added :  Pizza    The Unthinkable Pizza    CategoryBest Sellers       Size Regular
Added :  Pizza no 1    Category 2    S

Added :  Pizza no 8    Category 2    Size 1
Added :  Pizza    Chicken Sausage    CategoryBest Sellers       Size Regular
Added :  Pizza no 8    Category 2    Size 1
Added :  Pizza    Chicken Sausage    CategoryBest Sellers       Size Regular
Added :  Pizza no 8    Category 2    Size 1
Added :  Pizza    Chicken Sausage    CategoryBest Sellers       Size Regular
Added :  Pizza no 8    Category 2    Size 1
Added :  Pizza    Chicken Sausage    CategoryBest Sellers       Size Regular
------------------------------
Pizza Section No :  2
Actual Section No :  4
No. of items :  18
Added :  Pizza no 1    Category 4    Size 3
Added :  Pizza    The Unthinkable Pizza    CategoryVeg Pizza       Size Large
Added :  Pizza no 1    Category 4    Size 2
Added :  Pizza    The Unthinkable Pizza    CategoryVeg Pizza       Size Medium
Added :  Pizza no 1    Category 4    Size 2
Added :  Pizza    The Unthinkable Pizza    CategoryVeg Pizza       Size Medium
Added :  Pizza no 1    Category 4    Size 2
Added :  

Added :  Pizza no 8    Category 4    Size 1
Added :  Pizza    Deluxe Veggie    CategoryVeg Pizza       Size Regular
Added :  Pizza no 8    Category 4    Size 1
Added :  Pizza    Deluxe Veggie    CategoryVeg Pizza       Size Regular
Added :  Pizza no 8    Category 4    Size 1
Added :  Pizza    Deluxe Veggie    CategoryVeg Pizza       Size Regular
Added :  Pizza no 8    Category 4    Size 1
Added :  Pizza    Deluxe Veggie    CategoryVeg Pizza       Size Regular
Added :  Pizza no 9    Category 4    Size 3
Added :  Pizza    Veg Extravaganza    CategoryVeg Pizza       Size Large
Added :  Pizza no 9    Category 4    Size 2
Added :  Pizza    Veg Extravaganza    CategoryVeg Pizza       Size Medium
Added :  Pizza no 9    Category 4    Size 2
Added :  Pizza    Veg Extravaganza    CategoryVeg Pizza       Size Medium
Added :  Pizza no 9    Category 4    Size 2
Added :  Pizza    Veg Extravaganza    CategoryVeg Pizza       Size Medium
Added :  Pizza no 9    Category 4    Size 2
Added :  Pizza    Veg

Added :  Pizza no 16    Category 4    Size 1
Added :  Pizza    Achari Do Pyaza    CategoryVeg Pizza       Size Regular
Added :  Pizza no 16    Category 4    Size 1
Added :  Pizza    Achari Do Pyaza    CategoryVeg Pizza       Size Regular
Added :  Pizza no 16    Category 4    Size 1
Added :  Pizza    Achari Do Pyaza    CategoryVeg Pizza       Size Regular
Added :  Pizza no 16    Category 4    Size 1
Added :  Pizza    Achari Do Pyaza    CategoryVeg Pizza       Size Regular
Added :  Pizza no 17    Category 4    Size 3
Added :  Pizza    Kadhai Paneer    CategoryVeg Pizza       Size Large
Added :  Pizza no 17    Category 4    Size 2
Added :  Pizza    Kadhai Paneer    CategoryVeg Pizza       Size Medium
Added :  Pizza no 17    Category 4    Size 2
Added :  Pizza    Kadhai Paneer    CategoryVeg Pizza       Size Medium
Added :  Pizza no 17    Category 4    Size 2
Added :  Pizza    Kadhai Paneer    CategoryVeg Pizza       Size Medium
Added :  Pizza no 17    Category 4    Size 2
Added :  Pizza  

Added :  Pizza no 6    Category 6    Size 1
Added :  Pizza    Chicken Golden Delight    CategoryNon-Veg Pizza       Size Regular
Added :  Pizza no 7    Category 6    Size 3
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Large
Added :  Pizza no 7    Category 6    Size 2
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Medium
Added :  Pizza no 7    Category 6    Size 2
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Medium
Added :  Pizza no 7    Category 6    Size 2
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Medium
Added :  Pizza no 7    Category 6    Size 2
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Medium
Added :  Pizza no 7    Category 6    Size 1
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Regular
Added :  Pizza no 7    Category 6    Size 1
Added :  Pizza    Chicken Pepperoni    CategoryNon-Veg Pizza       Size Regular
Added :  

Added :  Pizza no 1    Category 8    Size 2
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Medium
Added :  Pizza no 1    Category 8    Size 2
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Medium
Added :  Pizza no 1    Category 8    Size 2
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Medium
Added :  Pizza no 1    Category 8    Size 2
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Medium
Added :  Pizza no 1    Category 8    Size 1
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Regular
Added :  Pizza no 1    Category 8    Size 1
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Regular
Added :  Pizza no 1    Category 8    Size 1
Added :  Pizza    Spiced Double Chicken    CategoryChicken Lovers Pizza       Size Regular
Added :  Pizza no 1    Category 8    Size 1
Added :  Pizza 

Added :  Item    Everyday Value 2 Med Deluxe Paneer Combo    CategoryMeals & Combos       Size N/A
Added :  Item    Everyday Value 2 Med Classic Mix Combo    CategoryMeals & Combos       Size N/A
Added :  Item    Ultimate Cheese Burst Deluxe Combo (Veg)    CategoryMeals & Combos       Size N/A
Added :  Item    Ultimate Cheese Burst Premium Combo (Veg)    CategoryMeals & Combos       Size N/A
Added :  Item    Ultimate Cheese Burst Premium Combo (N Veg)    CategoryMeals & Combos       Size N/A
Added :  Item    Ultimate Cheese Burst Deluxe Combo (N Veg)    CategoryMeals & Combos       Size N/A
Added :  Item    Everyday Value 2 Reg N.Veg Deluxe Combo    CategoryMeals & Combos       Size N/A
Added :  Item    Everyday Value 2 Reg Veg Premium Combo    CategoryMeals & Combos       Size N/A
Added :  Item    Everyday Value 2 Reg N.Veg Premium Combo    CategoryMeals & Combos       Size N/A
Added :  Item    Everyday Value 2 Med Veg Combo    CategoryMeals & Combos       Size N/A
Added :  Item    Ev

In [5]:
pd.DataFrame(data).to_csv("Final_Data_Dominos.csv")